In [1]:
from __future__ import print_function
%load_ext autoreload
%autoreload 2
from src.scrap import *
from src.load import *
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

Find average price of each sector in 2019
Find average price of each sector for certain floor
Find average price of each sea

In [2]:
hdbdata = load_HDB()
hdbdata['remaining_lease']=pd.to_datetime(hdbdata['month']).dt.year - hdbdata['lease_commence_date']

In [3]:
def sector_mean(hdbdata):
    hdb_rec_group = hdbdata.groupby(hdbdata['pc'].apply(lambda x:int(x[0:2])))
    hdbdata_mean = hdb_rec_group['resale_price'].mean()
    return hdbdata_mean

In [4]:
def plot_mean(hdbSeries):
    shapefile = Path('/home/jovyan/shared-notebooks/notebooks/HDB_analysis/data/processed/sectorcode/sectorcode.shp')
    sgshp = geopandas.read_file(shapefile)
    sgshp_3857 = sgshp.to_crs(epsg=3857)
    sale_counts_shp = pd.concat([sgshp_3857.set_index('sectorcode'),hdbSeries],axis=1)
    plt.close('all')
    fig = plt.figure(figsize=(30, 10))
    ax = fig.add_subplot()
    sale_counts_shp.plot(ax=ax,
                         column='resale_price',
                         alpha=0.9,
                         edgecolor='k',
                         missing_kwds={'color': 'lightgrey', 'alpha':0.1},
                         legend=True,
                         )
    ctx.add_basemap(ax=ax, source=ctx.providers.Stamen.TonerLite)
    ax.set_axis_off()
    ax.set_title('Average price')

In [5]:
filterdict = {
'flat_model': widgets.SelectMultiple(
    options=hdbdata['flat_model'].unique(),
    value=list(hdbdata['flat_model'].unique()),
    description='Flat Model'
    ),
'flat_type': widgets.SelectMultiple(
    options=hdbdata['flat_type'].unique(),
    value=list(hdbdata['flat_type'].unique()),
    description='Flat Type'
    ),
'floor_area_sqm': widgets.FloatRangeSlider(
    value=[hdbdata['floor_area_sqm'].min(), hdbdata['floor_area_sqm'].max()],
    min=hdbdata['floor_area_sqm'].min(),
    max=hdbdata['floor_area_sqm'].max(),
    step=1,
    description='Floor Area (sqm):'
    ),
'remaining_lease': widgets.FloatRangeSlider(
    value=[hdbdata['remaining_lease'].min(), hdbdata['remaining_lease'].max()],
    min=hdbdata['remaining_lease'].min(),
    max=hdbdata['remaining_lease'].max(),
    step=1,
    description='Remaining Lease (years):'
    ),
'month': widgets.SelectionRangeSlider(
    options=[(i,i) for i in hdbdata['month'].unique()],
    index=(0, hdbdata['month'].unique().size-1),
    description='Date of transaction',
    ),
'storey_range': widgets.SelectMultiple(
    options=hdbdata['storey_range'].unique(),
    value=list(hdbdata['storey_range'].unique()),
    description='Storeys'
    )
}

In [104]:
@widgets.interact_manual(**filterdict)
def filter_HDB(flat_model, flat_type, floor_area_sqm, remaining_lease, month, storey_range):
    bool_array = hdbdata['pc']==hdbdata['pc']
    bool_array = bool_array & hdbdata['flat_model'].isin(flat_model)
    bool_array = bool_array & hdbdata['flat_type'].isin(flat_type)
    bool_array = bool_array & (hdbdata['floor_area_sqm'] <= floor_area_sqm[1]) & (hdbdata['floor_area_sqm'] >= floor_area_sqm[0])
    bool_array = bool_array & (hdbdata['month'] <= month[1]) & (hdbdata['month'] >= month[0])
    bool_array = bool_array & (hdbdata['remaining_lease'] <= remaining_lease[1]) & (hdbdata['remaining_lease'] >= remaining_lease[0])
    bool_array = bool_array & hdbdata['storey_range'].isin(storey_range)
    hdbdata_fil = hdbdata[bool_array]
    sm = sector_mean(hdbdata_fil)
    plot_mean(sm)
    return hdbdata_fil

# For Troubleshooting
#hdbdata_fil_widget = interactive(filter_HDB, {'manual': True}, **filterdict)
#display(hdbdata_fil_widget)
#hdbdata_fil = hdbdata_fil_widget.result

interactive(children=(SelectMultiple(description='Flat Model', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

In [ ]:
filterdict_results = {k : v.value for k,v in filterdict.items()}
pp = filter_HDB(**filterdict_results)
pp
hdbdata.columns
hdbdata[hdbdata.isnull()['floor_area_sqm']]